In [1]:
import mne
import warnings
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.exceptions import ConvergenceWarning
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

from mne.decoding import CSP

from utils.Pipeline import FourierTransform, Float64CSP, EEGStandardScaler, WaveletTransform
from utils.Preprocessing import load_and_preprocess_data


In [2]:
mne.set_log_level("CRITICAL")

# Suppress warnings from LogisticRegression and CSP
warnings.filterwarnings("ignore", category=UserWarning, module='sklearn.linear_model._logistic')
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=UserWarning, module='mne')

In [3]:
# Define subjects and runs for training and testing
train_subjects = [57, 22, 86]
train_tasks = {
    'Task 1': [3, 7, 11],
}

test_subjects = [34]
test_tasks = {
    'Task 1': [3]
}

In [4]:
#task_1 (open and close left or right fist)
train_runs = train_tasks['Task 1']
train_epochs = load_and_preprocess_data(train_subjects, train_runs)

X_train = train_epochs.get_data().astype('float64')
y_train = train_epochs.events[:, -1]

display(X_train.shape)
display(y_train.shape)


test_runs = test_tasks['Task 1']
test_epochs = load_and_preprocess_data(test_subjects, test_runs)

X_test = test_epochs.get_data().astype('float64')
y_test = test_epochs.events[:, -1]

display(X_test.shape)
display(y_test.shape)

(195, 64, 481)

(195,)

(22, 64, 481)

(22,)

In [5]:
# Flatten the data
X_train_flattened = X_train.reshape(X_train.shape[0], -1)
X_test_flattened = X_test.reshape(X_test.shape[0], -1)


display(X_train_flattened.shape)
display(X_train.shape)
display(X_test_flattened.shape)

(195, 30784)

(195, 64, 481)

(22, 30784)

---


## StandardScaler + PCA + Logistic Regression

In [6]:
std_pca_dslr = Pipeline([
    ('standarize', StandardScaler()),
    ('pca', PCA(n_components=25)),
    ('dslr', LogisticRegression(max_iter=10000))
])

display(std_pca_dslr)



Pipeline(steps=[('standarize', StandardScaler()), ('pca', PCA(n_components=25)),
                ('dslr', LogisticRegression(max_iter=10000))])

In [7]:
# Train the model

std_pca_dslr.fit(X_train_flattened, y_train)

# Evaluate the model
train_accuracy = std_pca_dslr.score(X_train_flattened, y_train)
test_accuracy = std_pca_dslr.score(X_test_flattened, y_test)
cv_scores = cross_val_score(std_pca_dslr, X_train_flattened, y_train, cv=5)

print(f"Train accuracy: {train_accuracy:.2f}")
print(f"Test accuracy: {test_accuracy:.2f}")
print(f"Each fold cross validation accuracy : {cv_scores}")
print(f"Mean cross validation accuracy : {np.mean(cv_scores):.2f}")

Train accuracy: 0.90
Test accuracy: 0.64
Each fold cross validation accuracy : [0.74358974 0.79487179 0.84615385 0.87179487 0.71794872]
Mean cross validation accuracy : 0.79


-----

## Fourier transform + CSP + Sub Vector Machine

In [8]:
fft_csp_svm = Pipeline([
    ('fft', FourierTransform()), 
    ('csp', CSP(n_components=4, reg=None, log=True)),  # Common Spatial Patterns
    ('svm', SVC(kernel='linear', C=1.0))  # Clasificador SVM con kernel lineal
])

display(fft_csp_svm)

Pipeline(steps=[('fft', FourierTransform()),
                ('csp',
                 CSP({'component_order': 'mutual_info',
 'cov_est': 'concat',
 'cov_method_params': None,
 'log': True,
 'n_components': 4,
 'norm_trace': False,
 'rank': None,
 'reg': None,
 'transform_into': 'average_power'})),
                ('svm', SVC(kernel='linear'))])

In [9]:
fft_csp_svm.fit(X_train, y_train)

train_accuracy = fft_csp_svm.score(X_train, y_train)
test_accuracy = fft_csp_svm.score(X_test, y_test)
cv_scores = cross_val_score(fft_csp_svm, X_train, y_train, cv=5)

print(f"Train accuracy: {train_accuracy:.2f}")
print(f"Test accuracy: {test_accuracy:.2f}")
print(f"Each fold cross validation accuracy : {cv_scores}")
print(f"Mean cross validation accuracy : {np.mean(cv_scores):.2f}")

Train accuracy: 0.76
Test accuracy: 0.64
Each fold cross validation accuracy : [0.51282051 0.64102564 0.46153846 0.64102564 0.51282051]
Mean cross validation accuracy : 0.55


------

## Fourier transform + EEG Custom Standard scaler + Custom CSP + Linear discriminant analysis

In [10]:
fourier_csp_lda = Pipeline([
        ('fourier', FourierTransform()),     
        ('scaler', EEGStandardScaler()),
        ('CSP', Float64CSP(n_components=4, reg=None, log=True)),
        ('LDA', LinearDiscriminantAnalysis())
    ])

display(fourier_csp_lda)

fourier_csp_lda.fit(X_train, y_train)

train_accuracy = fourier_csp_lda.score(X_train, y_train)
test_accuracy = fourier_csp_lda.score(X_test, y_test)
cv_scores = cross_val_score(fourier_csp_lda, X_train, y_train, cv=5)

print(f"Train accuracy: {train_accuracy:.2f}")
print(f"Test accuracy: {test_accuracy:.2f}")
print(f"Each fold cross validation accuracy : {cv_scores}")
print(f"Mean cross validation accuracy : {np.mean(cv_scores):.2f}")

Pipeline(steps=[('fourier', FourierTransform()),
                ('scaler', EEGStandardScaler()),
                ('CSP',
                 Float64CSP({'component_order': 'mutual_info',
 'cov_est': 'concat',
 'cov_method_params': None,
 'log': True,
 'n_components': 4,
 'norm_trace': False,
 'rank': None,
 'reg': None,
 'transform_into': 'average_power'})),
                ('LDA', LinearDiscriminantAnalysis())])

Train accuracy: 0.66
Test accuracy: 0.64
Each fold cross validation accuracy : [0.64102564 0.61538462 0.61538462 0.66666667 0.66666667]
Mean cross validation accuracy : 0.64


---

## Fourier + Standard scaler + CSP + SVM

In [11]:
fourier_sca_csp_svm = Pipeline([
        ('fourier', FourierTransform()),
        ('scaler', EEGStandardScaler()),
        ('CSP', Float64CSP(n_components=4, reg=None, log=True)),
        ('SVM', SVC(kernel='linear'))
    ])

display(fourier_sca_csp_svm)

fourier_sca_csp_svm.fit(X_train, y_train)

train_accuracy = fourier_sca_csp_svm.score(X_train, y_train)
test_accuracy = fourier_sca_csp_svm.score(X_test, y_test)

cv_scores = cross_val_score(fourier_sca_csp_svm, X_train, y_train, cv=5)

print(f"Train accuracy: {train_accuracy:.2f}")
print(f"Test accuracy: {test_accuracy:.2f}")
print(f"Each fold cross validation accuracy : {cv_scores}")
print(f"Mean cross validation accuracy : {np.mean(cv_scores):.2f}")

Pipeline(steps=[('fourier', FourierTransform()),
                ('scaler', EEGStandardScaler()),
                ('CSP',
                 Float64CSP({'component_order': 'mutual_info',
 'cov_est': 'concat',
 'cov_method_params': None,
 'log': True,
 'n_components': 4,
 'norm_trace': False,
 'rank': None,
 'reg': None,
 'transform_into': 'average_power'})),
                ('SVM', SVC(kernel='linear'))])

Train accuracy: 0.65
Test accuracy: 0.64
Each fold cross validation accuracy : [0.64102564 0.64102564 0.64102564 0.64102564 0.66666667]
Mean cross validation accuracy : 0.65


---

## Wavelet transform + Random forest classifier

In [12]:
wavelet_rf = Pipeline([
    ('wavelet', WaveletTransform(wavelet='db4', level=4)),
    ('rf', RandomForestClassifier(n_estimators=100))
])

display(wavelet_rf)

wavelet_rf.fit(X_train, y_train)

train_accuracy = wavelet_rf.score(X_train, y_train)
test_accuracy = wavelet_rf.score(X_test, y_test)
cv_scores = cross_val_score(wavelet_rf, X_train, y_train, cv=5)

print(f"Train accuracy: {train_accuracy:.2f}")
print(f"Test accuracy: {test_accuracy:.2f}")
print(f"Each fold cross validation accuracy : {cv_scores}")
print(f"Mean cross validation accuracy : {np.mean(cv_scores):.2f}")


Pipeline(steps=[('wavelet',
                 <utils.Pipeline.WaveletTransform object at 0x7373efb19e50>),
                ('rf', RandomForestClassifier())])

Train accuracy: 1.00
Test accuracy: 0.64
Each fold cross validation accuracy : [0.66666667 0.79487179 0.69230769 0.79487179 0.64102564]
Mean cross validation accuracy : 0.72


---


## Common space patterns + Random forest classifier

In [13]:
csp_rf = Pipeline([
    ('csp', Float64CSP(n_components=4, reg=None, log=True)),
    ('rf', RandomForestClassifier(n_estimators=100))
])

display(csp_rf)

csp_rf.fit(X_train, y_train)

train_accuracy = csp_rf.score(X_train, y_train)
test_accuracy = csp_rf.score(X_test, y_test)

cv_scores = cross_val_score(csp_rf, X_train, y_train, cv=5)

print(f"Train accuracy: {train_accuracy:.2f}")
print(f"Test accuracy: {test_accuracy:.2f}")
print(f"Each fold cross validation accuracy : {cv_scores}")
print(f"Mean cross validation accuracy : {np.mean(cv_scores):.2f}")

Pipeline(steps=[('csp',
                 Float64CSP({'component_order': 'mutual_info',
 'cov_est': 'concat',
 'cov_method_params': None,
 'log': True,
 'n_components': 4,
 'norm_trace': False,
 'rank': None,
 'reg': None,
 'transform_into': 'average_power'})),
                ('rf', RandomForestClassifier())])

Train accuracy: 1.00
Test accuracy: 0.64
Each fold cross validation accuracy : [0.58974359 0.58974359 0.48717949 0.51282051 0.61538462]
Mean cross validation accuracy : 0.56


---


## Fourier transform + Random forest

In [14]:
ft_rf = Pipeline([
    ('ft', FourierTransform()),
    ('rf', RandomForestClassifier(n_estimators=100))
])

display(ft_rf)

ft_rf.fit(X_train_flattened, y_train)

train_accuracy = ft_rf.score(X_train_flattened, y_train)
test_accuracy = ft_rf.score(X_test_flattened, y_test)

cv_scores = cross_val_score(ft_rf, X_train_flattened, y_train, cv=5)

print(f"Train accuracy: {train_accuracy:.2f}")
print(f"Test accuracy: {test_accuracy:.2f}")
print(f"Each fold cross validation accuracy : {cv_scores}")
print(f"Mean cross validation accuracy : {np.mean(cv_scores):.2f}")

Pipeline(steps=[('ft', FourierTransform()), ('rf', RandomForestClassifier())])

Train accuracy: 1.00
Test accuracy: 0.36
Each fold cross validation accuracy : [0.61538462 0.58974359 0.64102564 0.66666667 0.64102564]
Mean cross validation accuracy : 0.63


---


## Wavelet transform + Logistic regression

In [15]:
wavelet_lr = Pipeline([
    ('wavelet', WaveletTransform(wavelet='db4', level=4)),
    ('dslr', LogisticRegression(max_iter=10000))
])

display(wavelet_lr)

wavelet_lr.fit(X_train, y_train)

train_accuracy = wavelet_lr.score(X_train, y_train)
test_accuracy = wavelet_lr.score(X_test, y_test)

cv_scores = cross_val_score(wavelet_lr, X_train, y_train, cv=5)

print(f"Train accuracy: {train_accuracy:.2f}")
print(f"Test accuracy: {test_accuracy:.2f}")
print(f"Each fold cross validation accuracy : {cv_scores}")
print(f"Mean cross validation accuracy : {np.mean(cv_scores):.2f}")

Pipeline(steps=[('wavelet',
                 <utils.Pipeline.WaveletTransform object at 0x7373ef99a510>),
                ('dslr', LogisticRegression(max_iter=10000))])

Train accuracy: 0.65
Test accuracy: 0.64
Each fold cross validation accuracy : [0.64102564 0.64102564 0.64102564 0.64102564 0.66666667]
Mean cross validation accuracy : 0.65


---


## Wavelet transform + SVM

In [16]:
wavelet_svm = Pipeline([
    ('wavelet', WaveletTransform(wavelet='db4', level=4)),
    ('svm', SVC(kernel='linear'))
])

display(wavelet_lr)

wavelet_svm.fit(X_train, y_train)

train_accuracy = wavelet_svm.score(X_train, y_train)
test_accuracy = wavelet_svm.score(X_test, y_test)

cv_scores = cross_val_score(wavelet_svm, X_train, y_train, cv=5)

print(f"Train accuracy: {train_accuracy:.2f}")
print(f"Test accuracy: {test_accuracy:.2f}")
print(f"Each fold cross validation accuracy : {cv_scores}")
print(f"Mean cross validation accuracy : {np.mean(cv_scores):.2f}")

Pipeline(steps=[('wavelet',
                 <utils.Pipeline.WaveletTransform object at 0x7373ef99a510>),
                ('dslr', LogisticRegression(max_iter=10000))])

Train accuracy: 0.65
Test accuracy: 0.64
Each fold cross validation accuracy : [0.64102564 0.64102564 0.64102564 0.64102564 0.66666667]
Mean cross validation accuracy : 0.65


---

## Standard scaler + Multilayer Perceptron Neural Network

In [17]:
std_mlp = Pipeline([
    ('scaler', StandardScaler()),            
    ('mlp', MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500))
])

display(std_mlp)

std_mlp.fit(X_train_flattened, y_train)

train_accuracy = std_mlp.score(X_train_flattened, y_train)
test_accuracy = std_mlp.score(X_test_flattened, y_test)

cv_scores = cross_val_score(std_mlp, X_train_flattened, y_train, cv=5)

print(f"Train accuracy: {train_accuracy:.2f}")
print(f"Test accuracy: {test_accuracy:.2f}")
print(f"Each fold cross validation accuracy : {cv_scores}")
print(f"Mean cross validation accuracy : {np.mean(cv_scores):.2f}")

Pipeline(steps=[('scaler', StandardScaler()),
                ('mlp',
                 MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500))])

Train accuracy: 1.00
Test accuracy: 0.64
Each fold cross validation accuracy : [0.76923077 0.76923077 0.87179487 0.84615385 0.82051282]
Mean cross validation accuracy : 0.82


---

## Standard scaler + PCA + Multilayer perceptron neural network

In [18]:
sca_pca_mlp = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=10)),
    ('mlp', MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=100))
])

display(sca_pca_mlp)

sca_pca_mlp.fit(X_train_flattened, y_train)

train_accuracy = sca_pca_mlp.score(X_train_flattened, y_train)
test_accuracy = sca_pca_mlp.score(X_test_flattened, y_test)
cv_scores = cross_val_score(sca_pca_mlp, X_train_flattened, y_train, cv=5)

print(f"Train accuracy: {train_accuracy:.2f}")
print(f"Test accuracy: {test_accuracy:.2f}")
print(f"Each fold cross validation accuracy : {cv_scores}")
print(f"Mean cross validation accuracy : {np.mean(cv_scores):.2f}")

Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=10)),
                ('mlp',
                 MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=100))])

Train accuracy: 0.99
Test accuracy: 0.73
Each fold cross validation accuracy : [0.58974359 0.82051282 0.64102564 0.87179487 0.71794872]
Mean cross validation accuracy : 0.73
